Dada una imagen de patrón, podemos utilizar la información anterior para calcular su pose, o cómo está situado el objeto en el espacio, como cómo se gira, cómo se desplaza, etc. Para un objeto plano, podemos asumir Z=0, de modo que el problema ahora es cómo se coloca la cámara en el espacio para ver nuestra imagen de patrón. Entonces, si sabemos cómo se encuentra el objeto en el espacio, podemos dibujar algunos diagramas 2D para simular el efecto 3D

In [1]:
import numpy as np
import cv2 as cv
import glob
# Load previously saved data


In [ ]:
with np.load('B.npz') as X:
    mtx, dist, _, _ = [X[i] for i in ('mtx','dist','rvecs','tvecs')]

Creamos una función, draw, que toma las esquinas del tablero de ajedrez (obtenidas usando cv.findChessboardCorners() ) y los puntos del eje para dibujar un eje 3D.

In [ ]:
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

Luego, como en el caso anterior, creamos criterios de terminación, puntos de objeto (puntos 3D de las esquinas del tablero de ajedrez) y puntos de eje. Los puntos del eje son puntos en el espacio 3D para dibujar el eje. Dibujamos un eje de longitud 3 (las unidades estarán en términos del tamaño del cuadrado de ajedrez ya que calibramos en función de ese tamaño). Entonces nuestro eje X se dibuja de (0,0,0) a (3,0,0), lo mismo para el eje Y. Para el eje Z, se dibuja de (0,0,0) a (0,0,-3). Negativo denota que se dirige hacia la cámara.

In [ ]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

Ahora, como es habitual, cargamos cada imagen. Busque una cuadrícula de 7x6. Si lo encontramos, lo refinamos con píxeles de subesquina. Luego, para calcular la rotación y la traslación, usamos la función cv.solvePnPRansac() . Una vez que tenemos esas matrices de transformación, las usamos para proyectar los puntos de nuestro eje al plano de la imagen

In [ ]:
for fname in glob.glob('left*.jpg'):
    img = cv.imread(fname)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, (7,6),None)
    if ret == True:
        corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        # Find the rotation and translation vectors.
        ret,rvecs, tvecs = cv.solvePnP(objp, corners2, mtx, dist)
        # project 3D points to image plane
        imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, mtx, dist)
        img = draw(img,corners2,imgpts)
        cv.imshow('img',img)
        k = cv.waitKey(0) & 0xFF
        if k == ord('s'):
            cv.imwrite(fname[:6]+'.png', img)
cv.destroyAllWindows()